In [1]:
using DIVAnd
using PyPlot
using NCDatasets
using Missings
using Interpolations

if VERSION >= v"0.7"
    using Random
    using DelimitedFiles
    using Statistics
    using Printf
    using FileIO
else
    using Compat: @info, @warn, range, cat
end

In [27]:
# Module where the output grid and the modules are defined
include("../src/emodnet_bio_grid.jl");
include("../src/make_traits_products.jl");

In [4]:
datafile = "/home/ctroupin/tmp/Emodnet-Bio/Traits/Benthos_Traits_ab.csv"
outputfile = joinpath(outputdirTraits, replace(basename(datafile), ".csv" => ".nc"))

"../output/traits/Benthos_Traits_ab.nc"

## Read data

In [5]:
if isfile(datafile)
    data, header = readdlm(datafile,',',header = true);
else
    @warn "Data file doesn't exist"
end
varlist = header[5:end];
ntraits = length(varlist);
@info "Working on $(ntraits) traits"

┌ Info: Working on 62 traits
└ @ Main In[5]:8


In [29]:
?readdlm

search: readdlm readdir



```
readdlm(source, T::Type; options...)
```

The columns are assumed to be separated by one or more whitespaces. The end of line delimiter is taken as `\n`.

# Examples

```jldoctest
julia> using DelimitedFiles

julia> x = [1; 2; 3; 4];

julia> y = [5; 6; 7; 8];

julia> open("delim_file.txt", "w") do io
           writedlm(io, [x y])
       end;

julia> readdlm("delim_file.txt", Int64)
4×2 Array{Int64,2}:
 1  5
 2  6
 3  7
 4  8

julia> readdlm("delim_file.txt", Float64)
4×2 Array{Float64,2}:
 1.0  5.0
 2.0  6.0
 3.0  7.0
 4.0  8.0

julia> rm("delim_file.txt")
```

---

```
readdlm(source, delim::AbstractChar, T::Type; options...)
```

The end of line delimiter is taken as `\n`.

# Examples

```jldoctest
julia> using DelimitedFiles

julia> x = [1; 2; 3; 4];

julia> y = [1.1; 2.2; 3.3; 4.4];

julia> open("delim_file.txt", "w") do io
           writedlm(io, [x y], ',')
       end;

julia> readdlm("delim_file.txt", ',', Float64)
4×2 Array{Float64,2}:
 1.0  1.1
 2.0  2.2
 3.0  3.3
 4.0  4.4

julia> rm("delim_file.txt")
```

---

```
readdlm(source; options...)
```

The columns are assumed to be separated by one or more whitespaces. The end of line delimiter is taken as `\n`. If all data is numeric, the result will be a numeric array. If some elements cannot be parsed as numbers, a heterogeneous array of numbers and strings is returned.

# Examples

```jldoctest
julia> using DelimitedFiles

julia> x = [1; 2; 3; 4];

julia> y = ["a"; "b"; "c"; "d"];

julia> open("delim_file.txt", "w") do io
           writedlm(io, [x y])
       end;

julia> readdlm("delim_file.txt")
4×2 Array{Any,2}:
 1  "a"
 2  "b"
 3  "c"
 4  "d"

julia> rm("delim_file.txt")
```

---

```
readdlm(source, delim::AbstractChar; options...)
```

The end of line delimiter is taken as `\n`. If all data is numeric, the result will be a numeric array. If some elements cannot be parsed as numbers, a heterogeneous array of numbers and strings is returned.

# Examples

```jldoctest
julia> using DelimitedFiles

julia> x = [1; 2; 3; 4];

julia> y = [1.1; 2.2; 3.3; 4.4];

julia> open("delim_file.txt", "w") do io
           writedlm(io, [x y], ',')
       end;

julia> readdlm("delim_file.txt", ',')
4×2 Array{Float64,2}:
 1.0  1.1
 2.0  2.2
 3.0  3.3
 4.0  4.4

julia> rm("delim_file.txt")

julia> z = ["a"; "b"; "c"; "d"];

julia> open("delim_file.txt", "w") do io
           writedlm(io, [x z], ',')
       end;

julia> readdlm("delim_file.txt", ',')
4×2 Array{Any,2}:
 1  "a"
 2  "b"
 3  "c"
 4  "d"

julia> rm("delim_file.txt")
```

---

```
readdlm(source, delim::AbstractChar, eol::AbstractChar; options...)
```

If all data is numeric, the result will be a numeric array. If some elements cannot be parsed as numbers, a heterogeneous array of numbers and strings is returned.

---

```
readdlm(source, delim::AbstractChar, T::Type, eol::AbstractChar; header=false, skipstart=0, skipblanks=true, use_mmap, quotes=true, dims, comments=false, comment_char='#')
```

Read a matrix from the source where each line (separated by `eol`) gives one row, with elements separated by the given delimiter. The source can be a text file, stream or byte array. Memory mapped files can be used by passing the byte array representation of the mapped segment as source.

If `T` is a numeric type, the result is an array of that type, with any non-numeric elements as `NaN` for floating-point types, or zero. Other useful values of `T` include `String`, `AbstractString`, and `Any`.

If `header` is `true`, the first row of data will be read as header and the tuple `(data_cells, header_cells)` is returned instead of only `data_cells`.

Specifying `skipstart` will ignore the corresponding number of initial lines from the input.

If `skipblanks` is `true`, blank lines in the input will be ignored.

If `use_mmap` is `true`, the file specified by `source` is memory mapped for potential speedups. Default is `true` except on Windows. On Windows, you may want to specify `true` if the file is large, and is only read once and not written to.

If `quotes` is `true`, columns enclosed within double-quote (") characters are allowed to contain new lines and column delimiters. Double-quote characters within a quoted field must be escaped with another double-quote.  Specifying `dims` as a tuple of the expected rows and columns (including header, if any) may speed up reading of large files.  If `comments` is `true`, lines beginning with `comment_char` and text following `comment_char` in any line are ignored.

# Examples

```jldoctest
julia> using DelimitedFiles

julia> x = [1; 2; 3; 4];

julia> y = [5; 6; 7; 8];

julia> open("delim_file.txt", "w") do io
           writedlm(io, [x y])
       end

julia> readdlm("delim_file.txt", '\t', Int, '\n')
4×2 Array{Int64,2}:
 1  5
 2  6
 3  7
 4  8
```


### Get coordinates

In [6]:
indexlon = findfirst(header .== "x").I[2];
indexlat = findfirst(header .== "y").I[2];
obslon = Vector{Float64}(data[:,indexlon]);
obslat = Vector{Float64}(data[:,indexlat]);

### Mask and bathymetry

In [7]:
# Grid stored in emodnet_bio_grid.jl
xi,yi = DIVAnd.ndgrid(gridlonBenthos, gridlatBenthos);

# Mask
topodir = "../data/"
topofile = joinpath(topodir, "gebco_30sec_8.nc");

if isfile(topofile)
    bx, by, b = DIVAnd.load_bath(topofile,true,gridlonBenthos, gridlatBenthos);
    xmask, ymask, mmask = DIVAnd.load_mask(topofile,true,gridlonBenthos, gridlatBenthos,[0]);
else
    @error "Bathymetry file doesn't exist"
end
mmask = mmask[:,:,1]
@info size(mmask)

# Metrics
pm = ones(size(xi)) / (xi[2,1]-xi[1,1]);
pn = ones(size(xi)) / (yi[1,2]-yi[1,1]);

┌ Info: (451, 371)
└ @ Main In[7]:15


## Parameter choice 

In [10]:
# Correlation length
len = 4.;
# Observational error variance normalized by the background error variance
epsilon2 = 10.;

The weight is decreased for very close points.

In [11]:
rdiag=1.0./DIVAnd.weight_RtimesOne((obslon, obslat),(0.03,0.03))
@show maximum(rdiag),mean(rdiag);
epsilon2 = epsilon2*rdiag;

(maximum(rdiag), mean(rdiag)) = (439.41430029504994, 33.49833660679846)


## Analysis
All the fields are stored in a 3D array, to be later written to a netCDF file.

In [12]:
# Allocate matrix with all the gridded fields
trait_all = Array{Float64}(undef, ntraits, length(gridlonBenthos), length(gridlatBenthos));
trait_err = Array{Float64}(undef, length(gridlonBenthos), length(gridlatBenthos));
for i in 1:ntraits
    @info "Working on $(varlist[i])" 
    indextrait = findfirst(header .== varlist[i]).I[2];
    trait = Vector{Float64}(data[:,indextrait]) 
    traits_interp = traits_analysis(obslon, obslat, trait);
    if i == 1
        @info "Computing error field"
        trait_err = traits_error(obslon, obslat, trait);
    end
    trait_all[i,:,:] = traits_interp;
end

┌ Info: Working on Body.length..1cm
└ @ Main In[12]:5


 12.817391 seconds (27.17 M allocations: 2.641 GiB, 14.58% gc time)


┌ Info: Computing error field
└ @ Main In[12]:10
┌ Info: Working on Body.length.1.3cm
└ @ Main In[12]:5


  1.787046 seconds (2.30 M allocations: 1.450 GiB, 56.17% gc time)


┌ Info: Working on Body.length.3.10cm
└ @ Main In[12]:5


...

  1.847809 seconds (2.30 M allocations: 1.450 GiB, 56.02% gc time)


┌ Info: Working on Burrowing.depth.5.15cm
└ @ Main In[12]:5


  1.851872 seconds (2.30 M allocations: 1.450 GiB, 56.53% gc time)


┌ Info: Working on Burrowing.depth..15cm
└ @ Main In[12]:5


  1.809327 seconds (2.30 M allocations: 1.450 GiB, 56.19% gc time)


### Write netCDF

In [15]:
@info "Writing netCDF file $(outputfile)"
write_traits_nc(outputfile, gridlonBenthos, gridlatBenthos, 
                trait_all, trait_err, varlist);

┌ Info: Writing netCDF file ../output/traits/Benthos_Traits_ab.nc
└ @ Main In[15]:1


## Plots

In [16]:
for ii = 1:ntraits
    make_plot_trait(gridlonBenthos, gridlatBenthos, trait_all[ii,:,:], varlist[ii]; vmin=0)
    figname = joinpath(figdirTraits, replace(varlist[ii], "." => "_"))
    PyPlot.savefig(figname, dpi=300, bbox_inches="tight")
    PyPlot.close_figs()
end

## Repeat with the other file

In [17]:
datafile = "/home/ctroupin/tmp/Emodnet-Bio/Traits/Benthos_Traits_tx.csv"
outputfile = joinpath(outputdirTraits, replace(basename(datafile), ".csv" => ".nc"))
@info "Output file: $(outputfile)"

if isfile(datafile)
    data,header = readdlm(datafile,',',header = true);
else
    @warn "Data file doesn't exist"
end
varlist = header[5:end];
ntraits = length(varlist);
@info "Working on $(ntraits) traits"

indexlon = findfirst(header .== "x").I[2];
indexlat = findfirst(header .== "y").I[2];
obslon = Vector{Float64}(data[:,indexlon]);
obslat = Vector{Float64}(data[:,indexlat]);

# Correlation length
len = 2.;
# Observational error variance normalized by the background error variance
epsilon2 = 5.;

rdiag=1.0./DIVAnd.weight_RtimesOne((obslon, obslat),(0.03,0.03))
@show maximum(rdiag),mean(rdiag);
epsilon2 = epsilon2*rdiag;

# Allocate matrix with all the gridded fields
trait_tx_all = Array{Float64}(undef, ntraits, length(gridlonBenthos), length(gridlatBenthos));
trait_tx_err = Array{Float64}(undef, length(gridlonBenthos), length(gridlatBenthos));
for i in 1:ntraits
    @info "Working on $(varlist[i])" 
    indextrait = findfirst(header .== varlist[i]).I[2];
    trait = Vector{Float64}(data[:,indextrait]) 
    traits_interp = traits_analysis(obslon, obslat, trait);
    if i == 1
        @info "Computing error field"
        trait_tx_err = traits_error(obslon, obslat, trait);
    end
    trait_tx_all[i,:,:] = traits_interp;
end

┌ Info: Working on 62 traits
└ @ Main In[17]:11


(maximum(rdiag), mean(rdiag)) = (439.414300295079, 33.47717862234115)


┌ Info: Working on Body.length..1cm
└ @ Main In[17]:31


  2.210487 seconds (2.30 M allocations: 1.450 GiB, 62.93% gc time)


┌ Info: Computing error field
└ @ Main In[17]:36
┌ Info: Working on Body.length.1.3cm
└ @ Main In[17]:31


...

  1.900560 seconds (2.30 M allocations: 1.450 GiB, 58.29% gc time)


┌ Info: Working on Burrowing.depth.5.15cm
└ @ Main In[17]:31


  1.933883 seconds (2.30 M allocations: 1.450 GiB, 58.66% gc time)


┌ Info: Working on Burrowing.depth..15cm
└ @ Main In[17]:31


  1.952531 seconds (2.30 M allocations: 1.450 GiB, 57.38% gc time)


In [18]:
write_traits_nc(outputfile, gridlonBenthos, gridlatBenthos, 
                trait_tx_all, trait_tx_err, varlist);

## Fish traits

In [25]:
datafile = "/home/ctroupin/tmp/Emodnet-Bio/Traits/Fish_Traits.csv"
outputfile = joinpath(outputdirTraits, replace(basename(datafile), ".csv" => ".nc"))
@info "Output file: $(outputfile)"

┌ Info: Output file: ../output/traits/Fish_Traits.nc
└ @ Main In[25]:3


In [19]:
if isfile(datafile)
    data,header = readdlm(datafile,',',header = true);
else
    @warn "Data file doesn't exist"
end
varlist = header[5:end];
ntraits = length(varlist);
@info "Working on $(ntraits) traits"

indexlon = findfirst(header .== "x").I[2];
indexlat = findfirst(header .== "y").I[2];
obslon = Vector{Float64}(data[:,indexlon]);
obslat = Vector{Float64}(data[:,indexlat]);

# Grid stored in emodnet_bio_grid.jl
xi,yi = DIVAnd.ndgrid(gridlonFish, gridlatFish);

# Mask
topodir = "../data/"
topofile = joinpath(topodir, "gebco_30sec_8.nc");

if isfile(topofile)
    bx, by, b = DIVAnd.load_bath(topofile,true,gridlonFish, gridlatFish);
    xmask, ymask, mmask = DIVAnd.load_mask(topofile,true,gridlonFish, gridlatFish,[0]);
else
    @error "Bathymetry file doesn't exist"
end
mmask = mmask[:,:,1]
@info size(mmask)

# Metrics
pm = ones(size(xi)) / (xi[2,1]-xi[1,1]);
pn = ones(size(xi)) / (yi[1,2]-yi[1,1]);

# Correlation length
len = 2.;
# Observational error variance normalized by the background error variance
epsilon2 = 5.;

rdiag=1.0./DIVAnd.weight_RtimesOne((obslon, obslat),(0.03,0.03))
@show maximum(rdiag),mean(rdiag);
epsilon2 = epsilon2*rdiag;

# Allocate matrix with all the gridded fields
trait_fish_all = Array{Float64}(undef, ntraits, length(gridlonFish), length(gridlatFish));
trait_fish_err = Array{Float64}(undef, length(gridlonFish), length(gridlatFish));
for i in 1:ntraits
    @info "Working on $(varlist[i])" 
    indextrait = findfirst(header .== varlist[i]).I[2];
    trait = Vector{Float64}(data[:,indextrait]) 
    traits_interp = traits_analysis(obslon, obslat, trait);
    if i == 1
        @info "Computing error field"
        trait_fish_err = traits_error(obslon, obslat, trait);
    end
    trait_fish_all[i,:,:] = traits_interp;
end

┌ Info: Output file: ../output/fish/Fish_Traits.nc
└ @ Main In[19]:3
┌ Info: Working on 36 traits
└ @ Main In[19]:12
┌ Info: (391, 261)
└ @ Main In[19]:33


(maximum(rdiag), mean(rdiag)) = (34.37624664581306, 7.778784558619619)


┌ Info: Working on Size.10.20cm
└ @ Main In[19]:52


  1.002369 seconds (1.72 M allocations: 998.738 MiB, 46.69% gc time)


┌ Info: Computing error field
└ @ Main In[19]:57
┌ Info: Working on Size.20.50cm
└ @ Main In[19]:52


...

  1.018783 seconds (1.72 M allocations: 998.738 MiB, 46.13% gc time)


┌ Info: Working on Foraging.mode.Benthopelagic
└ @ Main In[19]:52


  1.011829 seconds (1.72 M allocations: 998.738 MiB, 46.09% gc time)


┌ Info: Working on Foraging.mode.Pelagic
└ @ Main In[19]:52


  1.011961 seconds (1.72 M allocations: 998.738 MiB, 45.93% gc time)


In [28]:
write_traits_nc(outputfile, gridlonFish, gridlatFish, 
                trait_fish_all, trait_fish_err, varlist);